<a href="https://colab.research.google.com/github/dscohen75/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/Copy_of_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [41]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv',
                 parse_dates=['SALE DATE'],
                 index_col='SALE DATE'
                )

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [42]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [43]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [5]:
top10

Index(['FLUSHING-NORTH', 'UPPER EAST SIDE (59-79)', 'UPPER EAST SIDE (79-96)',
       'BEDFORD STUYVESANT', 'BOROUGH PARK', 'UPPER WEST SIDE (59-79)',
       'GRAMERCY', 'ASTORIA', 'FOREST HILLS', 'UPPER WEST SIDE (79-96)'],
      dtype='object')

 - Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.
 - Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
 - Do one-hot encoding of categorical features.
 - Do feature selection with SelectKBest.
 - Fit a ridge regression model with multiple features. Use the normalize=True parameter (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)
 - Get mean absolute error for the test set.

In [13]:
df.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'SALE_DATE'],
      dtype='object')

In [44]:
type(df['SALE_PRICE'])

pandas.core.series.Series

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 23040 entries, 2019-01-01 to 2019-04-30
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         23040 non-null  object 
 1   NEIGHBORHOOD                    23040 non-null  object 
 2   BUILDING_CLASS_CATEGORY         23040 non-null  object 
 3   TAX_CLASS_AT_PRESENT            23039 non-null  object 
 4   BLOCK                           23040 non-null  int64  
 5   LOT                             23040 non-null  int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       23039 non-null  object 
 8   ADDRESS                         23040 non-null  object 
 9   APARTMENT_NUMBER                5201 non-null   object 
 10  ZIP_CODE                        23039 non-null  float64
 11  RESIDENTIAL_UNITS               23039 non-null  float64
 12  COMMERCIAL_UNIT

In [46]:
onefam = df[df['BUILDING_CLASS_CATEGORY']=='01 ONE FAMILY DWELLINGS']


In [47]:
onefam.shape

(5061, 20)

In [48]:
mask = (onefam['SALE_PRICE'] > 100000) & (onefam['SALE_PRICE'] < 2000000)

onefam = onefam.loc[mask]

onefam.shape

(3151, 20)

In [49]:
onefam.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE DATE,,,,,,,,,,,,,,,,,,,,
2019-01-01,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000
2019-01-01,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000
2019-01-02,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000


In [51]:
onefam['BOROUGH'].value_counts()

4    1580
5     738
3     537
2     293
1       3
Name: BOROUGH, dtype: int64

In [52]:
onefam['TAX_CLASS_AT_PRESENT'].value_counts()

1     3111
1D      40
Name: TAX_CLASS_AT_PRESENT, dtype: int64

In [53]:
onefam['BUILDING_CLASS_AT_PRESENT'].value_counts()

A1    1185
A5     985
A2     491
A9     239
A0      85
S1      48
A3      43
A8      40
A4      16
A6      15
B2       2
B3       1
S0       1
Name: BUILDING_CLASS_AT_PRESENT, dtype: int64

In [54]:
onefam['ADDRESS'].value_counts()

294 FREEBORN STREET    2
216-29 114TH ROAD      2
108-16 171ST PLACE     2
33 BAILEY PLACE        2
57 CHESTNUT STREET     2
                      ..
46-25 194TH STREET     1
84-24 57 AVENUE        1
3717 BARNES AVENUE     1
74 MCVEIGH AVE         1
10 FORD PLACE          1
Name: ADDRESS, Length: 3135, dtype: int64

In [55]:
onefam['LAND_SQUARE_FEET'].value_counts()

4,000    289
2,000    239
2,500    196
1,800    130
3,000    111
        ... 
2,068      1
4,325      1
1,674      1
5,279      1
3,555      1
Name: LAND_SQUARE_FEET, Length: 1035, dtype: int64

In [56]:
onefam['BUILDING_CLASS_AT_TIME_OF_SALE'].value_counts()

A1    1186
A5     985
A2     493
A9     239
A0      85
S1      48
A3      43
A8      40
A4      16
A6      15
S0       1
Name: BUILDING_CLASS_AT_TIME_OF_SALE, dtype: int64

In [57]:
onefam['TAX_CLASS_AT_TIME_OF_SALE'].value_counts()

1    3151
Name: TAX_CLASS_AT_TIME_OF_SALE, dtype: int64

In [66]:
onefam['ZIP_CODE'].value_counts()

10306.0    127
10312.0    124
10314.0    113
11434.0    100
11234.0     90
          ... 
11220.0      1
10459.0      1
11238.0      1
11102.0      1
11217.0      1
Name: ZIP_CODE, Length: 125, dtype: int64

In [59]:
onefam['LAND_SQUARE_FEET'].value_counts()


4,000    289
2,000    239
2,500    196
1,800    130
3,000    111
        ... 
2,068      1
4,325      1
1,674      1
5,279      1
3,555      1
Name: LAND_SQUARE_FEET, Length: 1035, dtype: int64

In [60]:
# LAND_SQUARE_FEET was read as strings.
# Remove symbols, convert to integer
onefam['LAND_SQUARE_FEET'] = (
    onefam['LAND_SQUARE_FEET'].str.replace(',','').astype(int))

In [61]:
onefam['LAND_SQUARE_FEET'].describe()

count     3151.000000
mean      3112.117423
std       1790.055434
min          0.000000
25%       2000.000000
50%       2542.000000
75%       4000.000000
max      18906.000000
Name: LAND_SQUARE_FEET, dtype: float64

In [89]:
# Remove certain columns because they are too high in cardinality, 
# Or all the same

target = 'SALE_PRICE'
y = onefam[target]
X = onefam.drop([target] + ['EASE-MENT','BUILDING_CLASS_CATEGORY', 'ADDRESS', 'APARTMENT_NUMBER', 'TAX_CLASS_AT_TIME_OF_SALE'], axis =1)

In [90]:
print(y.shape)
print(X.shape)

(3151,)
(3151, 14)


Split into training and validation sets:

In [91]:
# Use data from January — March 2019 to train. 
# Use data from April 2019 to test.

cutoff = '2019-04-01'
mask = onefam.index < cutoff

X_train, y_train = X.loc[mask], y.loc[mask]
X_val, y_val = X.loc[~mask], y.loc[~mask]


In [92]:
assert X_train.shape[0] + X_val.shape[0] == X.shape[0]

Establish a baseline

In [93]:
y_train.hist()
# why wouldn't this plot show?

In [94]:
y_train.mean()

621573.7423214999

In [95]:
from sklearn.metrics import mean_absolute_error

print('Baseline MAE:', mean_absolute_error(y_train, [y_train.mean()]*len(y_train)))

Baseline MAE: 214721.52773001452


Build Model

In [76]:
from sklearn.linear_model import LinearRegression
from category_encoders import OneHotEncoder

Tranform the categorical variables:

In [77]:
catvars = ['BOROUGH', 'NEIGHBORHOOD', 'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_PRESENT','BUILDING_CLASS_AT_TIME_OF_SALE']

In [96]:
# Instantiate transformer
transformer_1 = OneHotEncoder(use_cat_names=True, cols=catvars)

In [97]:
# Fit transformer to the data
transformer_1.fit(X_train)

# Transform the training data
XT_train = transformer_1.transform(X_train)

In [98]:
XT_train.shape

(2507, 46)

In [99]:
XT_train.columns

Index(['BOROUGH_3', 'BOROUGH_4', 'BOROUGH_2', 'BOROUGH_5', 'BOROUGH_1',
       'NEIGHBORHOOD_OTHER', 'NEIGHBORHOOD_FLUSHING-NORTH',
       'NEIGHBORHOOD_BEDFORD STUYVESANT', 'NEIGHBORHOOD_FOREST HILLS',
       'NEIGHBORHOOD_BOROUGH PARK', 'NEIGHBORHOOD_ASTORIA',
       'TAX_CLASS_AT_PRESENT_1', 'TAX_CLASS_AT_PRESENT_1D', 'BLOCK', 'LOT',
       'BUILDING_CLASS_AT_PRESENT_A9', 'BUILDING_CLASS_AT_PRESENT_A1',
       'BUILDING_CLASS_AT_PRESENT_A5', 'BUILDING_CLASS_AT_PRESENT_A0',
       'BUILDING_CLASS_AT_PRESENT_A2', 'BUILDING_CLASS_AT_PRESENT_A3',
       'BUILDING_CLASS_AT_PRESENT_S1', 'BUILDING_CLASS_AT_PRESENT_A4',
       'BUILDING_CLASS_AT_PRESENT_A6', 'BUILDING_CLASS_AT_PRESENT_A8',
       'BUILDING_CLASS_AT_PRESENT_B2', 'BUILDING_CLASS_AT_PRESENT_S0',
       'BUILDING_CLASS_AT_PRESENT_B3', 'ZIP_CODE', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'BUILDING_CLASS_AT_TIME_OF_SALE_A9',
       'BUILDING_CLASS

## That's a lot of columns!

## Select K Best:

In [84]:
from sklearn.feature_selection import SelectKBest

In [101]:
# Instantiate the transformer
transformer_2 = SelectKBest(k=15)

# Fit transformer to the training data
transformer_2.fit(XT_train, y_train)

# Transform the training data
XTT_train = transformer_2.transform(XT_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


## Predictor - Ridge Regression

In [104]:
from sklearn.linear_model import Ridge

In [107]:
# Instantiate model
predictor = Ridge(normalize=True)

# Fit model to training data
predictor.fit(XTT_train, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None, normalize=True,
      random_state=None, solver='auto', tol=0.001)

In [108]:
# Make predictions with the model 
# from the twice-transformed training data

y_pred = predictor.predict(XTT_train)

# Calculate MAE

print('Training MAE:', mean_absolute_error(y_train, y_pred))

Training MAE: 188161.74233083212


In [110]:
# Transform the validation data with the same 2 transformers

XT_val = transformer_1.transform(X_val)
XTT_val = transformer_2.transform(XT_val)

In [112]:
# Make predictions from the transformed validation data
# 

y_pred_val = predictor.predict(XTT_val)

# Calculate MAE
print('Training MAE:', mean_absolute_error(y_val, y_pred_val))

Training MAE: 184279.70903475504
